# Yandex ML Cup 2021, NLP Section

Baseline notebook

Adapted for Google Colab and extended by Denis Volk

https://contest.yandex.ru/yacup/contest/29253/problems/

## Настраиваем среду

In [1]:
!nvidia-smi

Sun Oct 17 11:24:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install transformers torch sentencepiece gensim

     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 31.9 MB/s 
     |████████████████████████████████| 3.3 MB 38.5 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 46.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import pandas as pd
import numpy as np

## Загружаем скачанный классификатор токсичности

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# path_ro_roberta = "unitary/multilingual-toxic-xlm-roberta"
path_to_roberta = "/content/drive/MyDrive/Yandex-MLCup-2021/nlp/trained_roberta"
tokenizer = AutoTokenizer.from_pretrained(path_to_roberta)

model = AutoModelForSequenceClassification.from_pretrained(path_to_roberta).cuda()

TOXIC_CLASS=-1
TOKENIZATION_TYPE='sentencepiece'


## Функции для применения классификатора

In [7]:
from torch import softmax, sigmoid
import numpy as np


ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))
ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)


def logits_to_toxic_probas(logits):
    if logits.shape[-1] > 1:
        activation = lambda x: softmax(x, -1)
    else:
        activation = sigmoid
    return activation(logits)[:, TOXIC_CLASS].cpu().detach().numpy()


def is_word_start(token):
    if TOKENIZATION_TYPE == 'sentencepiece':
        return token.startswith('▁')
    if TOKENIZATION_TYPE == 'bert':
        return not token.startswith('##')
    raise ValueError("Unknown tokenization type")


def normalize(sentence, max_tokens_per_word=20):
    sentence = ''.join(map(lambda c: c if c.isalpha() else ' ', sentence.lower()))
    ids = tokenizer(sentence)['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(ids)[1:-1]
    
    result = []
    num_continuation_tokens = 0
    for token in tokens:
        if not is_word_start(token):
            num_continuation_tokens += 1
            if num_continuation_tokens < max_tokens_per_word:
                result.append(token.lstrip('#▁'))
        else:
            num_continuation_tokens = 0
            result.extend([' ', token.lstrip('▁#')])
    
    return ''.join(result).strip()

def iterate_batches(data, batch_size=40):
    batch = []
    for x in data:
        batch.append(x)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

from tqdm.auto import tqdm
def predict_toxicity(sentences, batch_size=5, threshold=0.5, return_scores=False, verbose=True, device='cuda'):
    results = []
    tqdm_fn = tqdm if verbose else lambda x, total: x
    for batch in tqdm_fn(iterate_batches(sentences, batch_size), total=np.ceil(len(sentences) / batch_size)):
        normalized = [normalize(sent, max_tokens_per_word=5) for sent in batch]
        tokenized = tokenizer(normalized, return_tensors='pt', padding=True, max_length=512, truncation=True)
        
        logits = model.to(device)(**{key: val.to(device) for key, val in tokenized.items()}).logits
        preds = logits_to_toxic_probas(logits)
        if not return_scores:
            preds = preds >= threshold
        results.extend(preds)
    return results


## Читаем тестовый набор

In [8]:
texts = []
texts_raw = []
with open("/content/drive/MyDrive/Yandex-MLCup-2021/nlp/public_testset.txt", 'rt') as f:
    for line in f:
        texts.append(normalize(line)) 
        texts_raw.append((line, normalize(line)))

Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors


In [9]:
sample_sentence = texts_raw[57][0]
sample_sentence

'он здорово приложил Пугачеву в совместной книге с Тополем о путешествии по Франции.Видно,она сильно достала его тогда.\n'

In [10]:
sentence = ''.join(map(lambda c: c if c.isalpha() else ' ', sample_sentence.lower()))
sentence

'он здорово приложил пугачеву в совместной книге с тополем о путешествии по франции видно она сильно достала его тогда  '

In [11]:
ids = tokenizer(sentence)['input_ids']
ids

[0,
 1266,
 14217,
 197,
 162307,
 547,
 13572,
 680,
 26401,
 105,
 49,
 120549,
 312,
 176375,
 135,
 690,
 145466,
 130,
 407,
 12469,
 24860,
 22573,
 1993,
 129,
 125998,
 1993,
 62669,
 2732,
 38519,
 12281,
 551,
 1739,
 20878,
 6,
 2]

In [12]:
tokens = tokenizer.convert_ids_to_tokens(ids)[1:-1]
tokens

['▁он',
 '▁здоров',
 'о',
 '▁приложи',
 'л',
 '▁пу',
 'га',
 'чев',
 'у',
 '▁в',
 '▁совместно',
 'й',
 '▁книге',
 '▁с',
 '▁то',
 'поле',
 'м',
 '▁о',
 '▁пут',
 'еше',
 'ств',
 'ии',
 '▁по',
 '▁франц',
 'ии',
 '▁видно',
 '▁она',
 '▁сильно',
 '▁доста',
 'ла',
 '▁его',
 '▁тогда',
 '▁']

In [13]:
texts_raw[-10:]

[('дураки 🤗🤗🤗\n', 'дураки'),
 ('Губка хорошая амвеевская, а простая всё корябает\n',
  'губка хорошая амвеевская а простая всё корябает'),
 ('Кто начал возвышать воров банкиров, артистов педерастов, а рабочего человека опустили ниже плинтуса, все пошло от правительства от ихней сраной идиологии\n',
  'кто начал возвышать воров банкиров артистов педерастов а рабочего человека опустили ниже плинтуса все пошло от правительства от ихней сраной идиологии'),
 ('ей это пофигу  ей лишбы брехуть перед путином\n',
  'ей это пофигу ей лишбы брехуть перед путином'),
 ('предоставьте другие плиззз...\n', 'предоставьте другие плиззз'),
 ('во первых,я не азербайджанка.во вторых это не единственный у вас случай.учить историю надо.не ту которую отфильтровали в инете армяне.цыгане очень древняя нация.только сейчас их корни были найдены и страна из которой они вышли.вы одной крови.ничего личного,только факты.близкородственные браки у вас порицаются,но не близкородственные интимные  связи.вы не цыгане.близ

(denis) Вопросы:
* Зачем так сложно? почему нельзя просто удалить знаки препинания и привести к lowercase?

## Вычисляем токсичность отдельных слов

In [14]:
import torch

words = set()
for text in texts:
    words.update(text.split())
words = sorted(words)

with torch.inference_mode():
    word_toxicities = predict_toxicity(words, batch_size=100, return_scores=True)
    
toxicity = dict(zip(words, word_toxicities))


  0%|          | 0/221.0 [00:00<?, ?it/s]

In [15]:
word_toxicity_df = pd.DataFrame.from_dict({'word': words, 'toxicity': word_toxicities})

In [16]:
word_toxicity_df.sort_values(by='toxicity', ascending=False).head(20)

,word,toxicity
12400,педерастов,0.990629
19785,ублюдочных,0.990629
12721,пиндосов,0.990628
19784,ублюдок,0.990627
4806,ебанутых,0.990624
20263,уродуйбезкультурье,0.990622
2923,выродки,0.990622
4428,долбоящеры,0.990617
12720,пиндосии,0.990608
12697,пидорас,0.990603


## Ниже читаем эмбеддинги слов и описываем функции их обработки

In [62]:
import gensim
from pymystem3 import Mystem

stemmer = Mystem()

In [51]:
embs_file = np.load('/content/drive/MyDrive/Yandex-MLCup-2021/nlp/embeddings_with_lemmas.npz', allow_pickle=True)
embs_vectors = embs_file['vectors']
embs_vectors_normed = embs_vectors / np.linalg.norm(embs_vectors, axis=1, keepdims=True)
embs_voc = embs_file['voc'].item()

embs_voc_by_id = [None for i in range(len(embs_vectors))]
for word, idx in embs_voc.items():
    if embs_voc_by_id[idx] is None:
        embs_voc_by_id[idx] = word

In [52]:
embs_vectors_normed[:5]

array([[-0.00158465,  0.1170527 ,  0.01341355, ..., -0.06672466,
        -0.00114055, -0.02048047],
       [ 0.04194824,  0.06409481, -0.07620999, ..., -0.00977619,
        -0.02718164,  0.03142696],
       [ 0.06089458, -0.00213495, -0.03098529, ..., -0.01324698,
        -0.02739514,  0.05458483],
       [ 0.10831252,  0.08426785,  0.01629879, ..., -0.05433694,
        -0.01024846,  0.02216649],
       [ 0.02091573, -0.01429716, -0.04098631, ..., -0.05328086,
        -0.10346793,  0.05339317]], dtype=float32)

In [58]:
def get_w2v_indicies(a):
    res = []
    if isinstance(a, str):
        a = a.split()
    for w in a:
        print(f'w = {w}')
        if w in embs_voc:
            res.append(embs_voc[w])
        else:
            lemma = stemmer.lemmatize(w)[0]
            print(f'lemma = {lemma}')
            res.append(embs_voc.get(lemma, None))
    return res

def calc_embs(words):
    words = ' '.join(map(normalize, words))
    inds = get_w2v_indicies(words)
    return [None if i is None else embs_vectors[i] for i in inds]

Сложим эмбеддинги нетоксичных слов в kd-дерево, чтобы можно было близко искать ближайших соседей

In [54]:
toxicity_threshold = 0.5

In [22]:
nontoxic_emb_inds = [ind for word, ind in embs_voc.items() if toxicity.get(word, 1.0) <= toxicity_threshold]
embs_vectors_normed_nontoxic = embs_vectors_normed[nontoxic_emb_inds]

In [23]:
from sklearn.neighbors import KDTree
embs_tree = KDTree(embs_vectors_normed_nontoxic, leaf_size=20)

Функция находит самое близкое нетоксичное слово по предпосчитанным эмбеддингам слов

In [59]:
from functools import lru_cache

@lru_cache()
def find_closest_nontoxic(word, threshold=0.5, allow_self=False):
    print(f'Input word: {word} [{toxicity.get(word, 1.0)}]')
    if toxicity.get(word, 1.0) <= threshold:
        print(f'Good word!')
        return word
    
    if word not in toxicity and word not in embs_voc:
        print('Word not found.')
        return None
    
    threshold = min(toxicity.get(word, threshold), threshold)
    word = normalize(word)
    print(f'Normalized word: {word}')
    
    word_emb = calc_embs([word])
    if word_emb is None or word_emb[0] is None:
        print(f'Embedding not found.')
        return None
    
    for i in embs_tree.query(word_emb)[1][0]:
        other_word = embs_voc_by_id[nontoxic_emb_inds[i]]
        print(f'Output word: {other_word}')
        if (other_word != word or allow_self) and toxicity.get(other_word, 1.0) <= threshold:
            return other_word
    print('Returning None')
    return None

Заменяем токсичные слова на ближайшие по эмбеддингам не-токсичные

In [60]:
def detox(line):
    words = normalize(line).split()
    fixed_words = [find_closest_nontoxic(word, threshold=toxicity_threshold, allow_self=True) or '' for word in words]
    return ' '.join(fixed_words)

In [63]:
detox('Мама мыла раму')

Input word: мама [0.07839752733707428]
Good word!
Input word: мыла [1.0]
Word not found.
Input word: раму [1.0]
Word not found.


'мама  '

In [64]:
detox('В траве сидел кузнечик, зелёненький он был')

Input word: в [0.11217756569385529]
Good word!
Input word: траве [1.0]
Word not found.
Input word: сидел [0.13705702126026154]
Good word!
Input word: кузнечик [1.0]
Normalized word: кузнечик
w = кузнечик
Output word: муха
Input word: зелёненький [1.0]
Word not found.
Input word: он [0.05697011947631836]
Good word!
Input word: был [0.09250128269195557]
Good word!


'в  сидел муха  он был'

In [61]:
detox('сдохнет')

Input word: сдохнет [0.8175283670425415]
Normalized word: сдохнет
w = сдохнет


KeyboardInterrupt: ignored

In [31]:
detox('скоро сдохнет и все вернется')

KeyboardInterrupt: ignored

In [28]:
detox('Он скоро сдохнет')

KeyboardInterrupt: ignored

In [ ]:
detox('Он скоро сдохнет и все')

In [39]:
detox('Он скоро и все вернется')

'он скоро и все вернется'

In [ ]:
texts[0]

'он скоро сдохнет и все вернется'

In [ ]:
detox('Он скоро сдохнет и все вернется')

KeyboardInterrupt: ignored

In [ ]:
detox('Он скоро сдохнет и все вернется')

KeyboardInterrupt: ignored

In [ ]:
detox(texts[0])

BrokenPipeError: ignored

In [ ]:
len(texts)

2500

In [ ]:
fixed_texts = list(map(detox, tqdm(texts[:10])))
fixed_texts

  0%|          | 0/10 [00:00<?, ?it/s]

BrokenPipeError: ignored

запишем результат в файл

In [ ]:
with open('baseline_fixed.txt', 'wt') as f:
    for text in fixed_texts:
        print(text, file=f)

Скор, если никак не изменять комментарии:

In [ ]:
!python3.7 score.py public_testset.short.txt public_testset.short.txt  --embeddings embeddings_with_lemmas.npz --lm lm.binary --model ./trained_roberta/ --device cuda --score -

Скор бейзлайна:

In [ ]:
!python3.7 score.py public_testset.short.txt baseline_fixed.txt  --embeddings embeddings_with_lemmas.npz --lm lm.binary --model ./trained_roberta/ --device cuda --score -

Сохраним данные для бейзлайна online-задачи

In [ ]:
!mkdir -p online_baseline

In [ ]:
import pickle as pkl

with open('./online_baseline/data.pkl', 'wb') as f:
    pkl.dump(toxicity, f)
    pkl.dump(nontoxic_emb_inds, f)